In [1]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np

classifiedSentimentFile = "app_reviews_txt_short0_out.txt"

In [2]:
comments = pd.read_csv(classifiedSentimentFile, delimiter='\t', encoding='latin1')
comments.head()

,Positive,Negative,Text
0,2,-2,Every other (always the first time) I open the...
1,2,-4,Latest update is a huge dissapointment! The ap...
2,1,-2,Recent update has significantly change the app...
3,2,-4,On the latest update using your mobile it is t...
4,3,-3,The standards have really slipped lately. Now ...


In [3]:
label_sentistrength = []
#print(comments.iloc[5742])
for i,row in comments.iterrows():
    p = row[0]
    n = row[1]
    if p+n == 0:
        label_sentistrength.append(0)
    elif p+n<0:
        label_sentistrength.append(-1)
    elif p+n>0:
        label_sentistrength.append(1)
    if row[2] is np.NaN:
        comments.iloc[i,2] = 'N/A'
comments['label_sentistrength'] = label_sentistrength
#comments.head

In [4]:
comments.iloc[5740:5745]

,Positive,Negative,Text,label_sentistrength
5740,3,-2,Great experience. No stress for paying less.,1
5741,3,-1,Great app,1
5742,1,-1,N/A,0
5743,2,-1,Good so far,1
5744,3,-1,Great.,1


In [5]:
analyzer = SentimentIntensityAnalyzer()
label_vader = []
score_vader = []
threshold = 0.3
for i,row in comments.iterrows():
    vs = analyzer.polarity_scores(row[2])['compound']
    #print(i,' ',vs)
    score_vader.append(vs) 
    if vs > threshold:
        label_vader.append(1)
    elif vs < -1*threshold:
        label_vader.append(-1)
    else:
        label_vader.append(0)  

        
comments['score_vader'] = score_vader
comments['label_vader'] = label_vader
comments.head



<bound method NDFrame.head of        Positive  Negative                                               Text  \
0             2        -2  Every other (always the first time) I open the...   
1             2        -4  Latest update is a huge dissapointment! The ap...   
2             1        -2  Recent update has significantly change the app...   
3             2        -4  On the latest update using your mobile it is t...   
4             3        -3  The standards have really slipped lately. Now ...   
...         ...       ...                                                ...   
19995         1        -1                                      Easy booking.   
19996         2        -2  Apart from once where my booking appeared to b...   
19997         3        -1                                      Love this app   
19998         1        -1  Longtime user and no issues to date. Easy to n...   
19999         4        -1  I love how easily I can book without paying up...   

       la

In [6]:
print(comments.iloc[1,2])
print(analyzer.polarity_scores(comments.iloc[1,2]))


Latest update is a huge dissapointment! The app is incredibly slow to load, I haven't seen something so slow since Internet Explorer... In the previous version, the account dashboard was so easy to access, now it's like going through a maze to access anything... Trying to track your parcel to get an idea when it will be delivered? I think you've got better chances of catching 5G network on a flying plane. This "upgrade" was a total fail...
{'neg': 0.042, 'neu': 0.806, 'pos': 0.152, 'compound': 0.829}


In [7]:
result_vader = 0
for i in range(len(label_sentistrength)):
    if label_sentistrength[i] == label_vader[i]:
        result_vader += 1
print('the Accuracy of Vader: ', result_vader/len(label_sentistrength),'%')
print()

precision_pos = 0
vader_pos = 0
precision_neg = 0
vader_neg = 0
precision_neu = 0
vader_neu = 0
for i in range(len(label_sentistrength)):
    if label_vader[i] == 1:
        vader_pos += 1
        if label_sentistrength[i] == 1:
            precision_pos += 1  
            
    if label_vader[i] == -1:
        vader_neg += 1
        if label_sentistrength[i] == -1:
            precision_neg += 1
    if label_vader[i] == 0:
        vader_neu += 1
        if label_sentistrength[i] == 0:
            precision_neu += 1
print('the precision of positive: ', precision_pos/vader_pos,'%')
print('the precision of negative: ', precision_neg/vader_neg,'%')
print('the precision of neutral: ', precision_neu/vader_neu,'%')
print('the Precision of Vader: ', (precision_pos/vader_pos + precision_neg/vader_neg + precision_neu/vader_neu)/3,'%')
print()

pre_pos = precision_pos/vader_pos
pre_neg = precision_neg/vader_neg
pre_neu = precision_neu/vader_neu
pre = (precision_pos/vader_pos + precision_neg/vader_neg + precision_neu/vader_neu)/3


recall_pos = 0
vader_pos = 0
recall_neg = 0
vader_neg = 0
recall_neu = 0
vader_neu = 0
for i in range(len(label_sentistrength)):
    if label_sentistrength[i] == 1:
        vader_pos += 1
        if label_vader[i] == 1:
            recall_pos += 1  
            
    if label_sentistrength[i] == -1:
        vader_neg += 1
        if label_vader[i] == -1:
            recall_neg += 1
    if label_sentistrength[i] == 0:
        vader_neu += 1
        if label_vader[i] == 0:
            recall_neu += 1
print('the recall of positive: ', recall_pos/vader_pos,'%')
print('the recall of negative: ', recall_neg/vader_neg,'%')
print('the recall of neutral: ', recall_neu/vader_neu,'%')
print('the Recall of Vader: ', (recall_pos/vader_pos + recall_neg/vader_neg + recall_neu/vader_neu)/3,'%')
print()

rec_pos = recall_pos/vader_pos
rec_neg = recall_neg/vader_neg
rec_neu = recall_neu/vader_neu
rec = (recall_pos/vader_pos + recall_neg/vader_neg + recall_neu/vader_neu)/3

def cal_f1(pre, recall):
    return 2*pre*recall/(pre + recall)

print('the F1-score of positive: ', cal_f1(pre_pos,rec_pos))
print('the F1-score of negative: ', cal_f1(pre_neg,rec_neg))
print('the F1-score of neutral: ', cal_f1(pre_neu,rec_neu))
print('the F1-Score of Vader: ', cal_f1(pre,rec))

print()



the Accuracy of Vader:  0.67785 %

the precision of positive:  0.7181421198888448 %
the precision of negative:  0.6704831692237234 %
the precision of neutral:  0.6105812488752924 %
the Precision of Vader:  0.6664021793292869 %

the recall of positive:  0.826216031057319 %
the recall of negative:  0.5841979249800479 %
the recall of neutral:  0.5446227929373997 %
the Recall of Vader:  0.6516789163249221 %

the F1-score of positive:  0.7683975788467664
the F1-score of negative:  0.6243736005970787
the F1-score of neutral:  0.5757190124713667
the F1-Score of Vader:  0.6589583167435201



In [8]:
print(len(comments))
comments2 = comments.loc[comments['label_sentistrength'] != 0]
print(len(comments2))
print(comments2.head)

20000
13770
<bound method NDFrame.head of        Positive  Negative                                               Text  \
1             2        -4  Latest update is a huge dissapointment! The ap...   
2             1        -2  Recent update has significantly change the app...   
3             2        -4  On the latest update using your mobile it is t...   
5             2        -3  Used to be half decent but the last update mad...   
8             1        -4  Just downloaded the update after delaying as l...   
...         ...       ...                                                ...   
19992         2        -1  Always used it... Book without payment... Revi...   
19993         4        -1        Really nice app makes planning a trip easy!   
19994         2        -1                                               Good   
19997         3        -1                                      Love this app   
19999         4        -1  I love how easily I can book without paying up...  

In [9]:
analyzer2 = SentimentIntensityAnalyzer()
label_vader2 = []
score_vader2 = []
threshold = 0
for i,row in comments2.iterrows():
    vs = analyzer2.polarity_scores(row[2])['compound']
    #print(i,' ',vs)
    score_vader2.append(vs) 
    if vs > threshold:
        label_vader2.append(1)
    elif vs < -1*threshold:
        label_vader2.append(-1)
    else:
        label_vader2.append(0)  

        
comments2['score_vader2'] = score_vader2
comments2['label_vader2'] = label_vader2

result_vader2 = 0
for i,row in comments2.iterrows():
    if row[3] == row[7]:
        result_vader2 += 1
print('Accuracy of Vader: ', result_vader2/len(comments2))


precision_pos = 0
vader_pos = 0
precision_neg = 0
vader_neg = 0

for i in range(len(label_sentistrength)):
    if label_vader[i] == 1:
        vader_pos += 1
        if label_sentistrength[i] == 1:
            precision_pos += 1  
            
    if label_vader[i] == -1:
        vader_neg += 1
        if label_sentistrength[i] == -1:
            precision_neg += 1

print('the precision of positive: ', precision_pos/vader_pos,'%')
print('the precision of negative: ', precision_neg/vader_neg,'%')
print('the Precision of Vader: ', (precision_pos/vader_pos + precision_neg/vader_neg)/2,'%')
print()

pre_pos = precision_pos/vader_pos
pre_neg = precision_neg/vader_neg
pre = (precision_pos/vader_pos + precision_neg/vader_neg )/2


recall_pos = 0
vader_pos = 0
recall_neg = 0
vader_neg = 0

for i in range(len(label_sentistrength)):
    if label_sentistrength[i] == 1:
        vader_pos += 1
        if label_vader[i] == 1:
            recall_pos += 1  
            
    if label_sentistrength[i] == -1:
        vader_neg += 1
        if label_vader[i] == -1:
            recall_neg += 1

print('the recall of positive: ', recall_pos/vader_pos,'%')
print('the recall of negative: ', recall_neg/vader_neg,'%')
print('the Recall of Vader: ', (recall_pos/vader_pos + recall_neg/vader_neg)/2,'%')
print()

rec_pos = recall_pos/vader_pos
rec_neg = recall_neg/vader_neg
rec = (recall_pos/vader_pos + recall_neg/vader_neg)/2


print('the F1-score of positive: ', cal_f1(pre_pos,rec_pos))
print('the F1-score of negative: ', cal_f1(pre_neg,rec_neg))
print('the F1-Score of Vader: ', cal_f1(pre,rec))

print()


/var/folders/gz/_xm1hwt92wz9r9pfzfctvx6h0000gn/T/ipykernel_9821/3494575886.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments2['score_vader2'] = score_vader2
/var/folders/gz/_xm1hwt92wz9r9pfzfctvx6h0000gn/T/ipykernel_9821/3494575886.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments2['label_vader2'] = label_vader2


Accuracy of Vader:  0.8201161946259985
the precision of positive:  0.7181421198888448 %
the precision of negative:  0.6704831692237234 %
the Precision of Vader:  0.6943126445562842 %

the recall of positive:  0.826216031057319 %
the recall of negative:  0.5841979249800479 %
the Recall of Vader:  0.7052069780186834 %

the F1-score of positive:  0.7683975788467664
the F1-score of negative:  0.6243736005970787
the F1-Score of Vader:  0.6997174087017409



In [10]:
comments2.head


<bound method NDFrame.head of        Positive  Negative                                               Text  \
1             2        -4  Latest update is a huge dissapointment! The ap...   
2             1        -2  Recent update has significantly change the app...   
3             2        -4  On the latest update using your mobile it is t...   
5             2        -3  Used to be half decent but the last update mad...   
8             1        -4  Just downloaded the update after delaying as l...   
...         ...       ...                                                ...   
19992         2        -1  Always used it... Book without payment... Revi...   
19993         4        -1        Really nice app makes planning a trip easy!   
19994         2        -1                                               Good   
19997         3        -1                                      Love this app   
19999         4        -1  I love how easily I can book without paying up...   

       la